<a href="https://colab.research.google.com/github/AI4Finance-LLC/ElegantRL/blob/master/eRL_demo_StockTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [1]:
from elegantrl.run import *
from elegantrl.agent import AgentDQN
from elegantrl.envs.FinRL.StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf

# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [3]:
import pandas as pd

In [5]:
d = pd.read_pickle('StockTradingEnv_processed_data.df')

In [10]:
len(d)

5284

In [18]:
d1 = d[(d.date < '2021-01-01') & (d.date >= '2016-01-01')]

In [19]:
d2 = d[(d.date >= '2008-03-19') & (d.date < '2016-01-01')]

In [22]:
d

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2000-01-03,0.936384,1.004464,0.907924,0.860883,535796800,AAPL,0,0.000000,0.927237,0.721947,0.000000,-66.666667,100.000000,0.860883,0.860883,0.000000
1,2000-01-04,0.966518,0.987723,0.903460,0.788302,512377600,AAPL,1,-0.001628,0.927237,0.721947,0.000000,-66.666667,100.000000,0.824592,0.824592,0.000000
2,2000-01-05,0.926339,0.987165,0.919643,0.799837,778321600,AAPL,2,-0.001717,0.894348,0.738334,14.119612,-24.602649,100.000000,0.816341,0.816341,0.000000
3,2000-01-06,0.947545,0.955357,0.848214,0.730621,767972800,AAPL,3,-0.004215,0.901703,0.688118,7.524634,-133.333333,100.000000,0.794911,0.794911,0.000000
4,2000-01-07,0.861607,0.901786,0.852679,0.765229,460734400,AAPL,4,-0.004106,0.885194,0.692754,25.518884,-88.553561,100.000000,0.788974,0.788974,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,2020-12-24,131.320007,133.460007,131.100006,131.773087,54930100,AAPL,3,3.332375,133.084918,116.348350,61.315724,171.392618,31.194296,122.326539,119.069382,0.026951
5280,2020-12-28,133.990005,137.339996,133.509995,136.486053,124486200,AAPL,0,3.801722,134.689237,116.751032,64.485587,203.530539,40.801655,122.908336,119.403902,1.259002
5281,2020-12-29,138.050003,138.789993,134.339996,134.668762,121047300,AAPL,1,3.981151,135.718903,117.301006,62.444536,179.324440,43.929288,123.427893,119.770764,0.307219
5282,2020-12-30,135.580002,135.990005,133.399994,133.520477,96452100,AAPL,2,3.984758,136.575356,117.542911,61.178808,138.858632,38.845768,123.874559,120.060675,0.151054


In [2]:
# Agent
args = Arguments(if_on_policy=False)
args.agent = AgentDQN()

# Environment
tickers = ['AAPL']  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1
initial_capital = 1e6
initial_stocks = 0
buy_cost_pct = 1e-3
sell_cost_pct = 1e-3
start_date = '2008-03-19'
start_eval_date = '2016-01-01'
end_eval_date = '2021-01-01'

args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False)
args.env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True)

args.env.target_reward = 3
args.env_eval.target_reward = 3

# Hyperparameters
args.gamma = gamma
args.break_step = 10000
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = 1000
args.batch_size = 32
args.repeat_times = 4 # repeat_times * target_step == number of times we update before training
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = True
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)
args.target_step = 200 # number of exploration steps before training

| load data: .//StockTradingEnv_processed_data.df
processed df length: 5284
split results 1962
split results 1259
| price_ary.shape: (1962,), tech_ary.shape: (1962, 8), turbulence_ary.shape: (1962,)
| price_ary.shape: (1259,), tech_ary.shape: (1259, 8), turbulence_ary.shape: (1259,)
| load data: .//StockTradingEnv_processed_data.df
processed df length: 5284
split results 1962
split results 1259
| price_ary.shape: (1962,), tech_ary.shape: (1962, 8), turbulence_ary.shape: (1962,)
| price_ary.shape: (1259,), tech_ary.shape: (1259, 8), turbulence_ary.shape: (1259,)


# **Part 6: Train and Evaluate the Agent**

> The training and evaluating processes are all finished inside function **train_and_evaluate_mp()**, and the only parameter for it is **args**. It includes the fundamental objects in DRL:

*   agent,
*   environment.

> And it also includes the parameters for training-control:

*   batch_size,
*   target_step,
*   reward_scale,
*   gamma, etc.

> The parameters for evaluation-control:

*   break_step,
*   random_seed, etc.






In [3]:
train_and_evaluate(args) # the training process will terminate once it reaches the target reward.

| GPU id: 0, cwd: ./AgentDQN/StockTradingEnv-v1_0
| Remove history
ID      Step      MaxR |    avgR      stdR       objA      objC |  avgS  stdS
start exploring before training
Begin updating network before training


AttributeError: 'numpy.ndarray' object has no attribute 'dim'

Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [ ]:
args = Arguments(if_on_policy=True)
args.agent = AgentPPO()
args.env = StockTradingEnv(cwd='./', if_eval=True)
args.if_remove = False
args.cwd = './AgentPPO/StockTradingEnv-v1_0'
args.init_before_training()

env.draw_cumulative_return(args, torch)